In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import torch.autograd as autograd
import torchvision
from google.protobuf import text_format
import onnx
import numpy as np
import os
from onnx_tf.backend import run_model
from matplotlib import pyplot as plt

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 10, kernel_size=3, stride=2)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5, stride=2)
        self.fc1 = nn.Linear(720, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(-1, 720)
        x = F.relu(self.fc1(x))
        return F.softmax(x, dim=1)

In [13]:
model = Net()
dummy_input = Variable(torch.randn(1, 3, 32, 32))
torch.onnx.export(model, dummy_input, "pb/torch_gh.pb", verbose=True, export_params=True)

graph(%1 : Float(1, 3, 32, 32)
      %2 : Float(10, 3, 3, 3)
      %3 : Float(10)
      %4 : Float(20, 10, 5, 5)
      %5 : Float(20)
      %6 : Float(10, 720)
      %7 : Float(10)) {
  %9 : UNKNOWN_TYPE = Conv[kernel_shape=[3, 3], strides=[2, 2], pads=[0, 0, 0, 0], dilations=[1, 1], group=1](%1, %2), uses = [[%10.i0]];
  %10 : Float(1, 10, 15, 15) = Add[broadcast=1, axis=1](%9, %3), uses = [%11.i0];
  %11 : Float(1, 10, 15, 15) = Relu(%10), uses = [%12.i0];
  %13 : UNKNOWN_TYPE = Conv[kernel_shape=[5, 5], strides=[2, 2], pads=[0, 0, 0, 0], dilations=[1, 1], group=1](%11, %4), uses = [[%14.i0]];
  %14 : Float(1, 20, 6, 6) = Add[broadcast=1, axis=1](%13, %5), uses = [%15.i0];
  %15 : Float(1, 20, 6, 6) = Relu(%14), uses = [%16.i0];
  %16 : Float(1, 720) = Reshape[shape=[-1, 720]](%15), uses = [%19.i0];
  %19 : Float(1, 10) = Gemm[alpha=1, beta=1, broadcast=1, transB=1](%16, %6, %7), uses = [%20.i0];
  %20 : Float(1, 10) = Relu(%19), uses = [%21.i0];
  %21 : Float(1, 10) = Softmax[axis=1

In [9]:
data = np.random.randn(1, 3, 32, 32).astype(np.float32)

In [10]:
input = Variable(torch.from_numpy(data))
model(input)

Variable containing:
 0.0982  0.0982  0.1117  0.0982  0.0982  0.0982  0.0982  0.0982  0.1026  0.0982
[torch.FloatTensor of size 1x10]

In [11]:
with open(os.path.join('pb/torch_gh.pb'), "rb") as f:
    test = onnx.load(f)
    # data = np.transpose(data, (0, 2, 3, 1))
    rs_onnx = run_model(test, [data])
    print(rs_onnx)

/Users/wenhao/Projects/onnx-tensorflow/onnx_tf/backend.py:643: UserWarning: Unsupported kernel_shape attribute by Tensorflow in Conv operator. The attribute will be ignored.
  UserWarning)


ValueError: Cannot feed value of shape (1, 3, 32, 32) for Tensor '1_1:0', which has shape '(5, 3, 32, 32)'